In [51]:
import os
import re
import json
from itertools import chain

fpath_win = 'C:\\Users\\rwmil\\Documents\\GitHub\\llm\\llm\\'
fpath_mac = '/Users/richardmiller/Documents/GitHub/llm/'
os.chdir(fpath_win)
print(os.getcwd())

C:\Users\rwmil\Documents\GitHub\llm\llm


In [3]:
with open('./LLMs-from-scratch-main/ch02/01_main-chapter-code/the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()
f.close()

split_text = [re.split(r'([,.:;?_!"()\']|--|\s)',i) for i in raw_text.split()]
split_text = [i.strip() for item in split_text for i in item if i not in ['',' ']]
split_text.extend(['<|endoftext|>','<|unk|>'])

split_text = sorted(list(set(split_text)))
vocab = {token:i for i,token in enumerate(split_text)}

In [71]:
class SimpleTokenizer:
    def __init__(self,vocab):
        ''' The vocab object must be a dictionary or 
        dictionary-like object and must be defined 
        outside of the tokenizer class. It also must
        have the form of {string:integer}'''
        self.string_to_int = vocab
        self.int_to_string = {i:s for s,i in vocab.items()}
        self.re_string_encoder = r'([,.:;?_!"()\']|--|\s)' #Leave these here because I can use them later to define the regex
        self.re_string_decoder = r'\s+([,.:;?!"()\'])' # strings that I want to use later. If my texts become more complex.

    def encoder(self,text):
        preprocess = re.split(self.re_string_encoder,text)
        preprocess = [i.strip() for i in preprocess if i not in ['',' ']]
        preprocess = [i if i in self.string_to_int else '<|unk|>' for i in preprocess]
        
        
        return [self.string_to_int[s] for s in preprocess]

    def decoder(self,ids):
        text = ' '.join([self.int_to_string[i] for i in ids])
        text = re.sub(self.re_string_decoder,r'\1',text)
        return text

tokenizer = SimpleTokenizer(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
te = " <|endoftext|> ".join((text1, text2))
out = tokenizer.encoder(te)
out2 = tokenizer.decoder(out)

In [73]:
print(out2)

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.
